In [10]:
import pymysql
from tkinter import *
from tkinter import messagebox
from tkinter.simpledialog import *
import random

## 함수 선언 부
def connectpymysql() :
    global conn, curr, window, canvas
    conn = pymysql.connect(host='localhost', user='root', password='dark##2993', db='KingHotDB', charset='utf8')
    curr = conn.cursor()

def closepymysql() :
    global conn, curr, window, canvas
    curr.close()
    conn.close()
    curr, conn = None, None

def randomColor() :
    COLORS = ["black",  "red", "green", "blue", "magenta", "orange", "brown", "maroon", "coral"]
    return random.choice(COLORS)

def clearMap() :
    global conn, curr, window, canvas
    canvas.destroy()
    canvas = Canvas(window, height=SCR_HEIGHT, width=SCR_WIDTH) 
    canvas.pack()

def displayRestaurant() :
    global conn, curr, window, canvas
    connectpymysql()

    sql = "SELECT restName, ST_AsText((ST_Buffer(restLocation, 3))) FROM Restaurant"
    curr.execute(sql)
    
    while True :
        row = curr.fetchone()
        if not row :
            break
        restName, gisStr = row
        start = gisStr.index('(')
        start = gisStr.index('(', start + 1)
        start += 1
        end = gisStr.index(')')
        gisStr = gisStr[start:end]      # "x y,x y,...."
        gisList = gisStr.split(',')         # ["x y", "x y", ....]
        newGisList = []
        for xyStr in gisList :
            x, y = xyStr.split(' ') # "x y"를 "x"와  "y"로 분리
            x, y = float(x), float(y) # 실수로 형 변환
            xyList = [ (x+90)*2, SCR_HEIGHT-(y+90)*2] # [x, y] : 화면 좌표(90,90 추가) 및 2배 확대
            newGisList.append(xyList) #[ [x,y], [x,y] .... ]

        myColor = randomColor()
        canvas.create_line(newGisList, fill=myColor, width=3)
        # 해당 위치에 글자 쓰기.
        tx, ty = xyList[0], xyList[1]+15 # 오른쪽 아래에 쓰기
        canvas.create_text([tx, ty],fill=myColor,text=restName.split(' ')[2]) # 0호점 만 쓰기.

    closepymysql()

def displayManager() :
    global conn, curr, window, canvas
    connectpymysql()

    sql = "SELECT ManagerName, ST_AsText(Area) FROM Manager ORDER BY ManagerName"
    curr.execute(sql)
    
    while True :
        row = curr.fetchone()
        if not row :
            break
        managerName, gisStr = row
        start = gisStr.index('(')
        start = gisStr.index('(', start + 1)
        start += 1
        end = gisStr.index(')')
        gisStr = gisStr[start:end]      # "x y,x y,...."
        gisList = gisStr.split(',')         # ["x y", "x y", ....]
        newGisList = []
        for xyStr in gisList :
            x, y = xyStr.split(' ') # "x y"를 "x"와  "y"로 분리
            x, y = float(x), float(y) # 실수로 형 변환
            xyList = [ (x+90)*2, SCR_HEIGHT-(y+90)*2] # [x, y] : 화면 좌표(90,90 추가) 및 2배 확대
            newGisList.append(xyList) #[ [x,y], [x,y] .... ]
        
        canvas.create_polygon(newGisList, fill=randomColor())

    closepymysql()

def displayRoad() :
    global conn, curr, window, canvas
    connectpymysql()

    sql = "SELECT RoadName, ST_AsText(ST_BUFFER(RoadLine,2)) FROM Road"
    curr.execute(sql)
    
    while True :
        row = curr.fetchone()
        if not row :
            break
        managerName, gisStr = row
        start = gisStr.index('(')
        start = gisStr.index('(', start + 1)
        start += 1
        end = gisStr.index(')')
        gisStr = gisStr[start:end]      # "x y,x y,...."
        gisList = gisStr.split(',')         # ["x y", "x y", ....]
        newGisList = []
        for xyStr in gisList :
            x, y = xyStr.split(' ') # "x y"를 "x"와  "y"로 분리
            x, y = float(x), float(y) # 실수로 형 변환
            xyList = [ (x+90)*2, SCR_HEIGHT-(y+90)*2] # [x, y] : 화면 좌표(90,90 추가) 및 2배 확대
            newGisList.append(xyList) #[ [x,y], [x,y] .... ]
        
        canvas.create_polygon(newGisList, fill=randomColor())
        
    closepymysql()   

def showResMan() :
    global conn, curr, window, canvas

    displayRestaurant() # 우선 지점을 출력
    
    connectpymysql()
    sql = "SELECT M.ManagerName, R.restName, ST_AsText((ST_Buffer(R.restLocation, 3))) FROM Restaurant R, Manager M"
    sql += " WHERE ST_Contains(M.area, R.restLocation) = 1 ORDER BY R.restName" # 체인점 순으로 정렬

    curr.execute(sql)

    saveRest = '' # 관리자가 중복된 체인점인지 체크.
    while True :
        row = curr.fetchone()
        if not row :
            break
        managerName, restName, gisStr = row
        start = gisStr.index('(')
        start = gisStr.index('(', start + 1)
        start += 1
        end = gisStr.index(')')
        gisStr = gisStr[start:end]      # "x y,x y,...."
        gisList = gisStr.split(',')         # ["x y", "x y", ....]
        newGisList = []
        for xyStr in gisList :
            x, y = xyStr.split(' ') # "x y"를 "x"와  "y"로 분리
            x, y = float(x), float(y) # 실수로 형 변환
            xyList = [ (x+90)*2, SCR_HEIGHT-(y+90)*2] # [x, y] : 화면 좌표(90,90 추가) 및 2배 확대
            newGisList.append(xyList) #[ [x,y], [x,y] .... ]

        myColor = randomColor()
        if saveRest == restName : # 중복 관리지역만 추가로 폴리곤으로 처리.
            canvas.create_polygon(newGisList, fill=myColor)

        # 해당 위치에 글자 쓰기.
        # 관리자가 2명인 경우에는 추가 관리자 이름에 뒤에 붙여서 쓰기.
        if saveRest == restName :
            tx, ty = xyList[0], xyList[1]+45 # 두번째 관리자. 아래아래에 쓰기
        else :
            tx, ty = xyList[0], xyList[1]+30 # 첫번째 관리자. 아래에 쓰기
            
        canvas.create_text([tx, ty],fill=myColor,text=managerName) # 관리자 이름만 추가로..
        saveRest = restName

    closepymysql()

def showNearest() :
    global conn, curr, window, canvas    

    baseRest = '왕매워 짬뽕 ' +  askstring('기준 체인점', '체인점 번호를 입력하세요') + '호점'
    
    connectpymysql()    
    sql = "SELECT ST_AsText(R2.restLocation), ST_Distance(R1.restLocation, R2.restLocation) "
    sql += " FROM Restaurant R1, Restaurant R2 "
    sql += " WHERE R1.restName='" + baseRest + "' "
    sql += " ORDER BY ST_Distance(R1.restLocation, R2.restLocation) "
    curr.execute(sql)

    row = curr.fetchone() # 첫번째(가장 가까운 거리)는 자신
    gisStr, distance = row  # gisStr은 "POINT(-80 -30)" 형식 
    start = gisStr.index('(')
    start += 1
    end = gisStr.index(')')
    gisStr = gisStr[start:end]      # "x y"
    x, y = list(map(float, gisStr.split(' '))) # [x y]
    baseXY = [ (x+90)*2, SCR_HEIGHT-(y+90)*2] 

    lineWidth = 20
    while True :
        row = curr.fetchone()
        if not row :
            break
        gisStr, distance = row
        start = gisStr.index('(')
        start += 1
        end = gisStr.index(')')
        gisStr = gisStr[start:end]      # "x y"
        x, y = list(map(float, gisStr.split(' '))) # [x y]
        targetXY = [ (x+90)*2, SCR_HEIGHT-(y+90)*2] 

        myColor = randomColor()
        if lineWidth < 0 :
            lineWidth = 0
        canvas.create_line([baseXY, targetXY], fill=myColor, width=lineWidth)
        lineWidth -= 5 # 선두께 감소..        
    
    closepymysql()

    displayRestaurant() # 지점을 출력 (위쪽에 보이려 제일 나중에 출력)

## 전역 변수부
conn, curr = None, None
window, canvas = None, None

SCR_WIDTH, SCR_HEIGHT = 360, 360

## 메인 코드부
window=Tk()
window.title("왕매워 짭뽕 Ver 0.1")
canvas = Canvas(window, height=SCR_HEIGHT, width=SCR_WIDTH) 
canvas.pack()

mainMenu = Menu(window)
window.config(menu=mainMenu)

gis1Menu = Menu(mainMenu)
mainMenu.add_cascade(label="GIS 데이터 보기", menu=gis1Menu)
gis1Menu.add_command(label="체인점 보기", command=displayRestaurant)
gis1Menu.add_command(label="관리자 보기", command=displayManager)
gis1Menu.add_command(label="도로 보기", command=displayRoad)
gis1Menu.add_separator()
gis1Menu.add_command(label="화면 지우기", command=clearMap)
gis1Menu.add_separator()

gis2Menu = Menu(mainMenu)
mainMenu.add_cascade(label="GIS 데이터 분석", menu=gis2Menu)
gis2Menu.add_command(label="관리자별 담당 체인점", command=showResMan)
gis2Menu.add_command(label="가장 가까운 체인점", command=showNearest)

window.mainloop()


In [6]:
import pymysql

# 전역변수 선언부
conn, curs = None, None
data1, data2, data3, data4, data5, data6 = "", "", "", "", "", ""
sql=""
row = None

# 메인 코드
conn = pymysql.connect(host="localhost", user="root", password="dark##2993", db="hanbitDB", charset="utf8")
curs = conn.cursor()
sql = "CREATE TABLE IF NOT EXISTS userTable (id char(4), userName char(15), email char(20), birthYear int)"
curs.execute(sql)

while (True):
    data1 = input("사용자 ID ==> ")
    if data1 == "":
        break
    data2 = input("사용자 이름 ==> ")
    data3 = input("사용자 이메일 ==> ")
    data4 = input("사용자 출생연도 ==> ")
    sql = "INSERT INTO userTable VALUES('" + data1 + "','" + data2 + "','" + data3 + "','" + data4 + "')"
    curs.execute(sql)
    print(sql)

conn.commit()
conn.close()

INSERT INTO userTable VALUES('aaa','AAAAA','aa.aa.com','1999')
INSERT INTO userTable VALUES('bbb','BBBBB','bb@bb.com','1888')


In [8]:
import pymysql

# 전역변수 선언부
conn, curs = None, None
data1, data2, data3, data4 = "", "", "", ""
sql=""
row = None

# 메인 코드
conn = pymysql.connect(host="localhost", user="root", password="dark##2993", db="hanbitDB", charset="utf8")
curs = conn.cursor()
sql = "SELECT * FROM userTable"
curs.execute(sql)

print("사용자 ID    사용자 이름    이메일      출생연도")
print("=============================================")

while (True):
    row = curs.fetchone()
    if row == None:
        break
    data1, data2, data3, data4 = row
    print("%5s   %15s   %15s   %d" % (data1, data2, data3, data4))

conn.close()

사용자 ID    사용자 이름    이메일      출생연도
  aaa             AAAAA         aa.aa.com   1999
  bbb             BBBBB         bb@bb.com   1888


In [14]:
import pymysql
from tkinter import *
from tkinter import messagebox


## 함수 선언부
def insertData() :
    con, cur = None, None
    data1, data2, data3, data4 = "", "", "", ""
    sql=""

    conn = pymysql.connect(host='localhost', user='root', password='dark##2993', db='hanbitDB', charset='utf8')
    cur = conn.cursor()

    data1 = edt1.get();    data2 = edt2.get();    data3 = edt3.get();    data4 = edt4.get()
    try :
        sql = "INSERT INTO userTable VALUES('" + data1 + "','" + data2 + "','" + data3 + "'," + data4 + ")"
        cur.execute(sql)
    except :
        messagebox.showerror('오류', '데이터 입력 오류가 발생함')
    else :
        messagebox.showinfo('성공', '데이터 입력 성공')
    conn.commit()
    conn.close()

def selectData() :
    strData1, strData2, strData3, strData4  = [], [], [], []
    conn = pymysql.connect(host='localhost', user='root', password='dark##2993', db='hanbitDB', charset='utf8')
    cur = conn.cursor()
    cur.execute("SELECT * FROM userTable")
    strData1.append("사용자ID");      strData2.append("사용자이름")
    strData3.append("이메일");         strData4.append("출생연도")
    strData1.append("-----------");    strData2.append("-----------")
    strData3.append("-----------");    strData4.append("-----------")
    while (True) :
        row = cur.fetchone()
        if row== None :
            break;
        strData1.append(row[0]);        strData2.append(row[1])
        strData3.append(row[2]);        strData4.append(row[3])

    listData1.delete(0,listData1.size() - 1);    listData2.delete(0,listData2.size() - 1)
    listData3.delete(0,listData3.size() - 1);    listData4.delete(0,listData4.size() - 1)
    for item1, item2, item3, item4 in zip(strData1, strData2, strData3, strData4 ):
        listData1.insert(END, item1);        listData2.insert(END, item2)
        listData3.insert(END, item3);        listData4.insert(END, item4)
    conn.close()    

## 메인 코드부
window = Tk()
window.geometry("600x300")
window.title("GUI 데이터 입력")

edtFrame = Frame(window);
edtFrame.pack()
listFrame = Frame(window)
listFrame.pack(side = BOTTOM,fill=BOTH, expand=1)

edt1= Entry(edtFrame, width=10);    edt1.pack(side=LEFT,padx=10,pady=10)
edt2= Entry(edtFrame, width=10);    edt2.pack(side=LEFT,padx=10,pady=10)
edt3= Entry(edtFrame, width=10);    edt3.pack(side=LEFT,padx=10,pady=10)
edt4= Entry(edtFrame, width=10);    edt4.pack(side=LEFT,padx=10,pady=10)

btnInsert = Button(edtFrame, text="입력", command = insertData)
btnInsert.pack(side=LEFT,padx=10,pady=10)
btnSelect = Button(edtFrame, text="조회", command =selectData )
btnSelect.pack(side=LEFT,padx=10,pady=10)

listData1 = Listbox(listFrame,bg = 'yellow');
listData1.pack(side=LEFT,fill=BOTH, expand=1)
listData2 = Listbox(listFrame,bg = 'yellow')
listData2.pack(side=LEFT,fill=BOTH, expand=1)
listData3 = Listbox(listFrame,bg = 'yellow')
listData3.pack(side=LEFT,fill=BOTH, expand=1)
listData4 = Listbox(listFrame,bg = 'yellow')
listData4.pack(side=LEFT,fill=BOTH, expand=1)

window.mainloop()


In [20]:
import pymysql

conn = pymysql.connect(host="localhost", user="root", password="dark##2993", database="payroll", charset="utf8")

curs = conn.cursor()
curs.execute("SELECT id, stname, course, fee FROM registation")
records = curs.fetchall()
print(records)

()


In [6]:
# https://www.youtube.com/watch?v=Jpf_G1D3TlE
# https://www.tutussfunny.com/python-crud-tkinter-step-by-step-pymysql-database/


import tkinter as tk
from tkinter import ttk, messagebox
import pymysql
# import pymysql.connector
from tkinter import *

def GetValue(event):
    e1.delete(0, END)
    e2.delete(0, END)
    e3.delete(0, END)
    e4.delete(0, END)
    row_id = listBox.selection()[0]
    select = listBox.set(row_id)
    e1.insert(0,select['id'])
    e2.insert(0,select['stname'])
    e3.insert(0,select['mobile'])
    e4.insert(0,select['salary'])

def Add():
   print("Add button pressed")
   studid = e1.get()
   studname = e2.get()
   coursename = e3.get()
   fee = e4.get()

   conn = pymysql.connect(host="localhost", user="root", password="dark##2993", db="workbench", charset="utf8")

   curs = conn.cursor()

   try:
      sql = "INSERT INTO  registation (id,stname,course,fee) VALUES (%s, %s, %s, %s)"
      val = (studid, studname, coursename, fee)
      print(sql)
   #    curs.execute(sql)
   #    conn.commit()

   #    lastid = curs.lastrowid
   #    messagebox.showinfo("information", "Employee inserted successfully...")

   #    e1.delete(0, END)
   #    e2.delete(0, END)
   #    e3.delete(0, END)
   #    e4.delete(0, END)
   #    e1.focus_set()

   except Exception as e:
      print(e)

   #    conn.rollback()
   #    conn.close()

def update():
    studid = e1.get()
    studname = e2.get()
    coursename = e3.get()
    fee = e4.get()
    conn=pymysql.connector.connect(host="localhost",user="root",password="dark##2993",database="payroll")
    curs=conn.cursor()
    try:
       sql = "Update  registation set empname= %s,course= %s,fee= %s where id= %s"
       val = (studname,coursename,fee,studid)
       curs.execute(sql, val)
       conn.commit()
       lastid = curs.lastrowid
       messagebox.showinfo("information", "Record Updateddddd successfully...")
       e1.delete(0, END)
       e2.delete(0, END)
       e3.delete(0, END)
       e4.delete(0, END)
       e1.focus_set()
    except Exception as e:
       print(e)
       conn.rollback()
       conn.close()
       
def delete():
    studid = e1.get()
    conn=pymysql.connector.connect(host="localhost",user="root",password="dark##2993",database="payroll")
    curs=conn.cursor()
    try:
       sql = "delete from registation where id = %s"
       val = (studid,)
       curs.execute(sql, val)
       conn.commit()
       lastid = curs.lastrowid
       messagebox.showinfo("information", "Record Deleteeeee successfully...")
       e1.delete(0, END)
       e2.delete(0, END)
       e3.delete(0, END)
       e4.delete(0, END)
       e1.focus_set()
    except Exception as e:
       print(e)
       conn.rollback()
       conn.close()

def show():
   conn = pymysql.connect(host="localhost", user="root", password="dark##2993", database="payroll", charset="utf8")

   curs = conn.cursor()
   curs.execute("SELECT id, stname, course, fee FROM registation")
   records = curs.fetchall()

   for i, (id, stname, course, fee) in enumerate(records, start=1):
      listBox.insert("", "end", values=(id, stname, course, fee))
   conn.close()

root = Tk()
root.geometry("800x500")

global e1
global e2
global e3
global e4

tk.Label(root, text="Employee Registation", fg="red", font=(None, 30)).place(x=300, y=5)
tk.Label(root, text="Employee ID").place(x=10, y=10)
Label(root, text="Employee Name").place(x=10, y=40)
Label(root, text="Mobile").place(x=10, y=70)
Label(root, text="Salary").place(x=10, y=100)

e1 = Entry(root)
e1.place(x=140, y=10)
e2 = Entry(root)
e2.place(x=140, y=40)
e3 = Entry(root)
e3.place(x=140, y=70)
e4 = Entry(root)
e4.place(x=140, y=100)

Button(root, text="Add",command = Add,height=3, width= 13).place(x=30, y=130)
Button(root, text="update",command = update,height=3, width= 13).place(x=140, y=130)
Button(root, text="Delete",command = delete,height=3, width= 13).place(x=250, y=130)

cols = ('id', 'stname', 'mobile','salary')
listBox = ttk.Treeview(root, columns=cols, show='headings' )

for col in cols:
    listBox.heading(col, text=col)
    listBox.grid(row=1, column=0, columnspan=2)
    listBox.place(x=10, y=200)
    
show()
listBox.bind('<Double-Button-1>',GetValue)
root.mainloop()

Add button pressed
INSERT INTO  registation (id,stname,course,fee) VALUES (%s, %s, %s, %s)
